# Text Classification using TextVectorization layer
> Multiclass text classification from scratchusing the new Keras TextVectorization layer

- toc: true
- badges: true
- comments: true
- categories: tensorflow, keras, googlecloud
- image: images/chart-preview.png

In [1]:
##### Copyright 2020 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Multiclass text classification from scratch

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This notebook contains a walkthrough of text classification from scratch, starting from a directory of plain text files (a common scenario in practice). The first part demonstrates sentiment analysis, using an IMDB dataset. We demonstrate multiclass text classification, using a dataset of Stack Overflow questions.

** TODO ** Also in the CL, update the title of the other text classification notebook to use transfer learning.

In [4]:
!pip install -q tf-nightly
import tensorflow as tf

ERROR:root:code for hash md5 was not found.
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 147, in <module>
    globals()[__func_name] = __get_hash(__func_name)
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 97, in __get_builtin_constructor
    raise ValueError('unsupported hash type ' + name)
ValueError: unsupported hash type md5
ERROR:root:code for hash sha1 was not found.
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 147, in <module>
    globals()[__func_name] = __get_hash(__func_name)
  File "/usr/local/Cellar/python@2/2.7.16/Frameworks/Python.framework/Versions/2.7/lib/python2.7/hashlib.py", line 97, in __get_builtin_constructor
    raise ValueError('unsupported hash type ' + name)
ValueError: unsupported hash 

In [5]:
import numpy as np
from tensorflow.keras import preprocessing
print(tf.__version__)

2.2.0-dev20200507


## Multiclass text classification

The second part of this notebook classifies Stack Overflow posts as one of the most used languages today, namely Java, Javascript, Python or C#. This is an example of *multiclass* classification, different from the binary classification showed in Part 1.

We will use a [public dataset about Stack Overflow questions](https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow) available in Google Cloud marketplace. You can explore the dataset in [BigQuery](https://cloud.google.com/bigquery/) just by following the instructions of the former link. In this notebook, you will build a model to predict the tags of questions from Stack Overflow, using a pre-processed table already built and coming from the BigQuery dataset. To keep things simple our pre-processed table includes questions containing 4 possible programming-related tags: Java, Javascript, Python or C#

As in Part 1, Part 2 of this notebook uses [tf.keras](https://www.tensorflow.org/guide/keras) to build and train models in TensorFlow, as well as some TensorFlow experimental features, like the `TextVectorization` layer for word splitting & indexing.

### Download the BigQuery dataset

BigQuery has a  public dataset that includes more than 17 million Stack Overflow questions. We are going to download some posts labeled as one of the four most used languages today: java, javascript, python and C#, but to make this a harder problem to our model, we have replaced every instance of that word with another less used language today (but well-known some decades ago) called `fortran`. Otherwise, it will be very easy for the model to detect that a post is a java-related post just by finding the word `java` on it.

You can access the pre-processed fortran-filled dataset as a tar file [here](PENDING). Each of the four labels has approximate 10k samples for training/eval and 10k samples for test.

Let's first import libraries to make Part 2 of this notebook independent from Part 1, and then download our pre-processed dataset.


In [6]:
!gsutil cp gs://tensorflow-blog-rnn/so_posts_4labels_blank_80k.tar.gz .
# TODO: MOVE DATASET TO TF.ORG
!tar -xf so_posts_4labels_blank_80k.tar.gz



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying gs://tensorflow-blog-rnn/so_posts_4labels_blank_80k.tar.gz...
/ [0 files][    0.0 B/ 29.0 MiB]-- [0 files][  2.1 MiB/ 29.0 MiB]\\ [0 files][ 13.4 MiB/ 29.0 MiB]|// [0 files][ 24.0 MiB/ 29.0 MiB]-- [1 files][ 29.0 MiB/ 29.0 MiB]
Operation completed over 1 objects/29.0 MiB.                                     


In [7]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', batch_size=batch_size, validation_split=0.2, subset='training', seed=42)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=42)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'test', batch_size=batch_size)

Found 40000 files belonging to 4 classes.
Using 32000 files for training.
Found 40000 files belonging to 4 classes.
Using 8000 files for validation.
Found 40000 files belonging to 4 classes.


In [8]:
import time
start = time.time()
for text_batch, label_batch in raw_train_ds:
    pass
end = time.time()
print(end - start)

8.353302001953125


In [9]:
import time
start = time.time()
for text_batch, label_batch in raw_train_ds:
    pass
end = time.time()
print(end - start)

2.060072183609009


In [10]:
raw_train_ds = raw_train_ds.cache()
import time
start = time.time()
for text_batch, label_batch in raw_train_ds:
    pass
end = time.time()
print(end - start)

2.046962022781372


In [11]:
import time
start = time.time()
for text_batch, label_batch in raw_train_ds:
    pass
end = time.time()
print(end - start)

0.14096736907958984


### Explore the data

The dataset comes pre-processed, by replacing key words java, javascript, C# or python by `fortran`. In total, there are 4 labels (classes).

Note again that, as in Part 1, we can evaluate tensors using `.numpy()`, thanks to the eager execution of TensorFlow 2:

In [12]:
import time

for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])

b'how do i find missing dates in a list of sorted dates? in blank how do i find all the missing days in a sorted list of dates?\n'
3
b'"find the sequences of numbers in the list? there is a task to find all the sequences of numbers in the list, then add them another list. for example, there is such a sequence of numbers in list ...  12222533343332...only numbers must appear in the resulting list like this 44 77 88 000 a prerequisite is that repeated numbers must stand side by side .for example, so ...  5122225333433325...5 should not fall into the resulting list because they are not near each other, respectively (not a sequence)..list&lt;integer&gt; toplist = new arraylist&lt;&gt;();.    list&lt;integer&gt; result = new arraylist&lt;&gt;();.    int count = 0;.    boolean flag = true;..    while (count &lt; toplist.size()){.        while (flag) {.            for (int j = count + 1; j &lt; toplist.size(); j++) {.                if (toplist.get(count).equals(toplist.get(j))) {.           

Each label is an integer value between 0 and 3, correponsing to one of our four labels (0 to 3):

### Prepare data for training

Since the data is pre-processed, we do not need to make any additional steps like removing HTML tags, as we did in Part 1 of this notebook.

We can go directly to instantiate our text vectorization layer (experimental feature). We are using this layer to normalize, split, and map strings to integers, so we set our `output_mode` to `int`. We also set the same constants as Part 1 for the model, like an explicit maximum `sequence_length`.

In [13]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_features = 5000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [14]:
# Make a text-only dataset (no labels) and call adapt
text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### Vectorize the data



In [15]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

The vectorization layer transforms each input word of a sentence into a numerical representation, i.e. a list of token indices or vocabulary, with size defined by `max_features` (5000). Note that the output size is fixed, truncated by `sequence_length` (500), regardless of how many tokens resulted from the previous step, and this will be the input to our model.

Let's take a moment to understand the output of the vectorization layer. The output of each sentence is fixed to 500 integers, as stated by `sequence_length`. It should be noted that most of the values are zero, and this is due to the fact that there is no corresponding token in our vocabulary.


In [16]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])

[  22   40    3  139  490 1037    6    5   53    9 1131 1037    6   16
   22   40    3  139   73    2  490  711    6    5 1131   53    9 1037
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

### Build the model

The input data consists of an array of integer-encoded vocabulary, with a fixed size. The labels to predict are between 0 and 3, so instead of using a binary classifier, we will use a softmax classifier. We compile the model with an Adam optimizer and a different loss function from Part 1 (Sparse categorical crossentropy).

One of the parameters of the embedding layer is `max_features+1`and not `max_features`, and the reason is to add an extra token for an unknown word to our vocabulary in the input string.


In [17]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype='int64')

x = layers.Embedding(max_features + 1, embedding_dim)(inputs)
x = layers.Bidirectional(layers.LSTM(128))(x)
predictions = layers.Dense(4, activation='softmax', name='predictions')(x)

model = tf.keras.Model(inputs, predictions)

model.compile(
    loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the model

Train the model by passing the `Dataset` object to the model's fit function. Set the number of epochs.

In [18]:
epochs = 5

# Fit the model using the train and test datasets.
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
  11/1000 [..............................] - ETA: 8:49 - loss: 1.3928 - accuracy: 0.2472

KeyboardInterrupt: 

In [ ]:
model.summary()

### Evaluate the model

And let's see how the model performs. Two values will be returned: loss (a number which represents our error, lower values are better), and accuracy.

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## Learn more

This notebook uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For a more advanced text classification tutorial using `tf.keras`, see the [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/). In this notebook, we also use some TensorFlow experimental features, like the `TextVectorization` layer for word splitting & indexing.